# Cloud Front with API Gateway Origin

This notebook can be used to configure an active-standby two region serverless API project. This includes the 
following:

* Creation of a common API key that can be installed in both regions. This is needed to ensure transparent
failover from the perspective of the service consumer.

## Library Code

In [ ]:
# SDK Imports
import boto3

cformation_east = boto3.client('cloudformation', region_name='us-east-1')
cformation_west = boto3.client('cloudformation', region_name='us-west-2')

gw_east = boto3.client('apigateway', region_name='us-east-1')
gw_west = boto3.client('apigateway', region_name='us-west-2')

In [ ]:
def get_stack_name(service, stage):
    return '{}-{}'.format(service,stage)

In [ ]:
def get_endpoint(cf_client, stack_name):
    response = cf_client.describe_stacks(
        StackName=stack_name
    )
    
    outputs = response['Stacks'][0]['Outputs']
    endpoint =  [d for d in outputs if d['OutputKey'] == 'ServiceEndpoint'][0]['OutputValue']
    return endpoint

In [ ]:
def get_plan_and_api_ids(gw_client, service, stage):
    response = gw_client.get_usage_plans()
    plans = response['items']
    stack_name = get_stack_name(service, stage)
    plan =  [d for d in plans if d['name'] == stack_name][0]
    plan_id = plan['id']
    api_stage = [d for d in plan['apiStages'] if d['stage'] == stage][0]
    api_id = api_stage['apiId']
    return plan_id, api_id
    

In [ ]:
import uuid

def generate_api_key():
    return str(uuid.uuid4())

In [ ]:
def create_api_key_and_add_to_plan(gw_client, key_name, plan_id):
    id = generate_api_key()
    
    create_key_response = gw_client.create_api_key(
        name=key_name,
        enabled=True,
        generateDistinctId=True,
        value=id
    )
    
    key_id = create_key_response['id']
    
    plan_key_response = gw_client.create_usage_plan_key(
        usagePlanId=plan_id,
        keyId=key_id,
        keyType='API_KEY'
    )
    
    return id, key_id

In [ ]:
# Create a key and add it to the usage plan?
# - create_api_key - need key id output
# - you can get the usage plan id and the api id via get_usage_plan and matching the plan with same name
#   as the stack
# - create_usage_plan_key associates the key to the plan: inputs are plan id, key id

## Application Context

In [ ]:
service = 'serverless-rest-api-with-dynamodb'
stage = 'pp1'

In [ ]:
east_endpoint = get_endpoint(cformation_east, get_stack_name(service, stage))
print east_endpoint

## Key Synchronization

This part of the notebook creates a common key for the gateway fronted app in both regions.

In [ ]:
# Get the plan id
plan_id, api_id = get_plan_and_api_ids(gw_east, service, stage)
print plan_id
print api_id

In [ ]:
key_val, key_id = create_api_key_and_add_to_plan(gw_east, 'xregion_key', plan_id)
print key_val
print key_id

## Clean Up

Clean up stuff  - useful while building this book

In [ ]:
response = gw_east.delete_usage_plan_key(
    usagePlanId=plan_id,
    keyId=key_id
)

print response

In [ ]:
response = gw_east.delete_api_key(
    apiKey=key_id
)

print response